In [87]:
'''
Import the requests and BeautifulSoup modules
'''
import requests
from bs4 import BeautifulSoup

BASE_URL = "https://www.hipo.ro/"
all_job_links = []

for index in range(1, 112):
    req = requests.get(BASE_URL + "locuri-de-munca/cautajob/" + str(index))
    soup = BeautifulSoup(req.content, 'html.parser')
    job_links = soup.select('.job-item .text-start>a')
    for job_link in job_links:
        url = BASE_URL + job_link['href']
        all_job_links.append(url)

In [105]:
'''
Get the title and the info of each job
'''
all_job_info = []
counter = 0

for job_link in all_job_links[:2200]:
    page_soup = BeautifulSoup(requests.get(job_link).content, 'html.parser')
    title_elem = page_soup.find('h1', class_="entry-title")

    if title_elem is None:
        continue

    title = title_elem.text
    info_soup = page_soup.find('div', class_="info-row")
    info_table_rows = info_soup.find_all('tr')

    info_dict = {}
    for row in info_table_rows:
        elems = row.find_all('td')
        key = elems[0].text
        value_obj = elems[1]
        if value_obj.find('li') is not None:
            value = [li.text.replace('\n', '').replace('\t', '') for li in value_obj.find_all('li')]
        else:
            value = value_obj.text.replace('\n', '').replace('\t', '')
        
        info_dict[key] = value
    
    all_job_info.append((title, info_dict))
    print(f"Job {counter} done")
    counter += 1

Job 0 done
Job 1 done
Job 2 done
Job 3 done
Job 4 done
Job 5 done
Job 6 done
Job 7 done
Job 8 done
Job 9 done
Job 10 done
Job 11 done
Job 12 done
Job 13 done
Job 14 done
Job 15 done
Job 16 done
Job 17 done
Job 18 done
Job 19 done
Job 20 done
Job 21 done
Job 22 done
Job 23 done
Job 24 done
Job 25 done
Job 26 done
Job 27 done
Job 28 done
Job 29 done
Job 30 done
Job 31 done
Job 32 done
Job 33 done
Job 34 done
Job 35 done
Job 36 done
Job 37 done
Job 38 done
Job 39 done
Job 40 done
Job 41 done
Job 42 done
Job 43 done
Job 44 done
Job 45 done
Job 46 done
Job 47 done
Job 48 done
Job 49 done
Job 50 done
Job 51 done
Job 52 done
Job 53 done
Job 54 done
Job 55 done
Job 56 done
Job 57 done
Job 58 done
Job 59 done
Job 60 done
Job 61 done
Job 62 done
Job 63 done
Job 64 done
Job 65 done
Job 66 done
Job 67 done
Job 68 done
Job 69 done
Job 70 done
Job 71 done
Job 72 done
Job 73 done
Job 74 done
Job 75 done
Job 76 done
Job 77 done
Job 78 done
Job 79 done
Job 80 done
Job 81 done
Job 82 done
Job 83 done
Jo

In [6]:
'''
Transform the data into a DataFrame with title and dictionary columns
'''
import pandas as pd

columns = ['title'] + list(column.lower().replace(":", "").replace(" ", "_") for column in all_job_info[0][1].keys())
data_table = []

for title, info in all_job_info:
    # Convert info dictionary to a list; elements that have lists are converted to multiple columns
    data_row = [title] + list(info.values())
    data_table.append(data_row)


df = pd.DataFrame(data_table, columns=columns)
df.to_csv('jobs.csv', index=False)

NameError: name 'all_job_info' is not defined

In [148]:
'''
Use DataFrame to train sklearn.tree model
'''
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

# Assuming you already have your DataFrame 'df'

# Drop any rows with missing values
df.dropna(inplace=True)

# Initialize label encoder
label_encoder = LabelEncoder()

df.apply(label_encoder.fit_transform)

# Encode categorical variables
for column in df.columns[1:]:
    if df[column].dtype == 'object':
        df[column] = label_encoder.fit_transform(df[column])
        # df[column] = df[column].apply(lambda x: label_encoder.fit_transform([x])[0])

# Define features and target variable
X = df.drop(columns=df.columns[0])  # Features
y = df[df.columns[0]]  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(max_depth=5, random_state=0)
decision_tree.fit(X_train, y_train)

# Predict titles for test data
predicted_titles = decision_tree.predict(X_test)

# Optionally, you can print or use the predicted titles array
print("Predicted titles:", predicted_titles)


      title  angajator                                            domeniu  \
0       851        150                                      [IT Software]   
1       857         54                         [IT Hardware, IT Software]   
2       294         26                                           [Sales ]   
3      1514         27                          [Management - Consulting]   
4       847         26                                      [IT Software]   
...     ...        ...                                                ...   
2190    706        184  [Contabilitate Finante, Secretariat - Administ...   
2191   1000        202                                        [Marketing]   
2192   1004        202                                        [Marketing]   
2193    994        202                            [Contabilitate Finante]   
2194   2023         78                [Hoteluri - Restaurante - Cafenele]   

         tip_job                  nivel_job          orase actualizat_la  \

TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['list']